# RaspberryPi aufsetzten für Projektarbeiten an der NTB

Erstellt am 26.6.2018 für Raspberry Pi 3 Model B.

### Installation
Mit SDformatter (geht auch für Windows10) die SD-Karte formatieren.

**Installation dauert über 1 Stunde!** Von [www.raspberrypi.org](http://www.raspberrypi.org) aktuelles NOOBS. „Network install only“ (nicht offline). Voraussetzung ist Netzwerk-Zugriff. Anschliessend auf SD-Karte kopieren/entpacken.

Im Klassenverbund das WLAN «gast-ntb» verwenden. Das tagesgültige Passwort ist im Intranet.

Dann SD-Karte in RaspberryPi einsetzen und starten. Zuerst Sprache Deutsch und Tastatur CH eingeben, dann WLAN verbinden (hierzu WLAN user/pwd eingeben, wofür die richtige Tastatur gewählt werden muss um ggf Sonderzeichen eingeben zu können). **Raspbian with PIXEL (Desktop Version)** ankreuzen, und „Installieren“ klicken. 

Nach Installation erfolgt ein Neustart und es ist ein Login erforderlich. Prüfe ob die richtige Tastatur konfiguriert wurde mit Eingabe „z“ und „y“.
Login: pi; Password: raspberry.

Nun konfigurieren wir den RaspberryPi über „Einstellungen > Raspberry-Pi-Konfiguration"

Folgende Einstellungen vornehmen:
* SSH aktivieren in  „Interface Option“ (oder "Advanced Options") unter "SSH" enable, somit können wir später remote über ssh zugreifen.
* Hier im Menü „Boot Option“ die Desktop-Option und autologin wählen (default). 
* Tastatur und Sprache kann ebenfalls hier konfiguriert werden (bereits gemacht).

Neues Passwort: um nicht jedes Mal das lange Passwort „raspberry“ mit der „y“ oder „z“ Unsicherheit eingeben zu müssen, ändern wir das Passwort zu „??passwort wählen??“ (Achtung Kleinschreibung beachten). Änderung über die Konsole. Für den user „pi“ (Standard):

    passwd

Und für den root-user:

    sudo passwd


Dann wird das System aktualisiert. Dieser Schritt empfiehlt sich allgemein bei OpenSource-SW; immer auf dem aktuellem Stand arbeiten um nicht Probleme zu lösen, welche bereits gelöst wurden. Zuerst die „system’s package list“ updaten, sodass im Folgenden die aktuellsten Version installiert werden. 

    sudo apt-get update
    
Aktuell heisst das wir im Folgenden mit Python 3.x arbeiten werden. Um Bibliotheken hierfür zu installieren muss darauf geachtet werden das die "3"-Version verwendet wird. Z.B. **pip3 anstatt pip** oder spyder3 anstatt spyder. Ohne "3" wird meist die Version für Python 2.x verwendet. **Der Raspberry benötigt für das System beide Versionen!** Standardmässig wird Python2.x verwendet. Erkennbar durch "python --version", oder man startet auf der Console "python" und führt die Division 3/2 durch. Python2.x liefert 1. Wird auf der Konsole "python3" gestartet ergibt 3/2 = 1.5.

Anschliessend wird einige Programme:

    sudo apt-get install install python3-numpy python3-matplotlib spyder3
    sudo pip3 install django bme680
    
Spyder zur Erstellung von Python-Skripts (Oberfläche ist wie MATLAB). Die Verbindung von Python und html wird mit Django realisiert. BME680 für Sensor. Für BME680-Beispiele siehe (https://learn.pimoroni.com/tutorial/sandyj/getting-started-with-bme680-breakout).
 

## File Transfer SFTP 
Um bestehende py-Skripts vom PC auf den RaspberryPi zu übertragen benötigen wir einen Remote-Zugriff. 
Hierzu verwenden wir das auf Windows installierte Tool «WinSCP».
Windowstaste drücken und «WinSCP» eingeben. **Port: 22 (wichtig, weil SFTP)**, Username: pi, Password: ??passwort wählen??.

Die IP-Adresse des RPI erhält man durch Eingabe auf dem RPI:

    ifconfig

Alternativen zu WinSCP sind «Bitvise SSH Client» oder "FileZilla".

## Remote Desktop
Für den Remote Desktop Zugriff müssen erst folgende Pakete in dieser Reihenfolge installiert werden:

    sudo apt-get install tightvncserver -y
    sudo apt-get install xrdp -y


Bei Versionen vor 2018 ist vorgängig eine Deinstallation notwendig:

    sudo apt-get remove xrdp vnc4server tightvncserver
   
Beim Remote Desktop Zugriff wird die englische Tastatur (km-0409.ini) geladen, wir wollen aber die schweizer Tastatur (km-0807.ini). Hierzu verbinden wir uns über Remote Desktop mit dem RPI (Name des RPI ist IP-Adresse, siehe "ifcongif"). Da wir Änderungen im System vornehmen müssen wir uns mit "root" angemelden (Administrator-Rechte). 

Auf dem PC kann nun Remote Desktop-Connection gestartet werden. Es öffnet sich ein Fenster, bei welchem das Modul "sesman.Xvnc" gewählt wird. Der Login lautet:

Username: root

Password: ??passwort wählen??


Dann öffnen wir den Dateimanager: Darstellung bei linker Spalte von „Ort“ auf „Verzeichnisbaum“ stellen. Navigieren in den Ordner `/etc/xrdp`.
Dort nimmt er standardmässig die Datei „km-00000409.ini“. Wir suchen im Internet die Datei „km-0807.ini“ und laden diese. Wir kopieren unter „Downloads“ die Datei in den Ordner `/etc/xrdp`. Dann umbenennen „km-0409.ini“  in „km-00000409_orginal.ini“ und umbenennen „km-0807.ini“  in „km-00000409.ini“.


## Serielle Schnittstelle für GPIO mit Subplatine
MODBUS-RTU basiert auf RS485 und kann mit dem Raspberry Pi über eine USB-RS485 Schnittstelle angeprochen werden. An der NTB wurde eine Subplatine erstellt. Hierfür müssen die Folgenden Anpassungen vorgenommen werden:

 - In der Datei ** /boot/config.txt ** Bluetooth deaktivieren und uart aktivieren. GPIO-Pins (14, 15) sind dann frei für seriellen Schnittstelle. Am Ende der Datei folgende zwei Befehle ergänzen: `dtoverlay=pi3-disable-bt` und `enable_uart=1`

 - In der Datei ** /boot/cmdline.txt ** die serielle Console deaktivieren indem Einträge entfernt werden, welche wie folgt aussehen: `console=serial0,115200` oder `console=ttyAMA0,115200`

Unter Menü kann der Texteditor verwendet werden, jedoch nur wenn mit "root" eingeloggt ist. Auf der Konsole kann der Editor „nano“ werdendet werden. Consolenbefehle sind:

    sudo nano /boot/config.txt

    sudo nano /boot/cmdline.txt

Anschliessend speichern und Neustarten:

    sudo reboot

Nun müsste das Bluetooth-Symbol rechts oben deaktiv sein. Die Schnittstelle ist über Python erreichbar mit *** "dev/ttyAMA0" """.

Nun können die installierten Pakete upgraden werden und die ungenützten Abhängigkeiten entfernt werden. Im Klassenverbund nicht eingeben, da es zu lange dauert.

    sudo apt-get dist-upgrade -y
    sudo reboot
    

## Django
Für Django verwenden wir das Template https://github.com/markstaler/pv4ing/blob/master/4i.rar, bei welchem alles Vorinstalliert ist.



Möchten wir selber ein Django-Projekt aufsetzen, so beginnt dies mit dem anlegen eines Projektverzeichnis "4i". In diesem Verzeichnis wir ein Django-Projekt eröffnen.

    django-admin startproject pv4ing .

Der Punkt braucht es, damit Django im aktuellen Verzeichnis installiert wird. Dann:

    python manage.py migrate

Mit Spyder öffnen wir folgende Dateien und führen Änderungen durch:

### setting.py

    INSTALLED_APPS = [....
    ....,
    'pv4ing',
    ]
    
    
    ALLOWED_HOSTS = [
    '127.0.0.1',
    'localhost',
    '[ip-Adresse]',
    'www.myWebSite.ch',
    ]
    


### urls.py

    from django.conf.urls import url
    from . import views

    urlpatterns = [
        url(r'^&', views.home),
        url(r'^admin/', admin.site.urls),
        ]
& sollte Dolarzeichen sein.

### views.py (neu anlegen)
    siehe Vorlage

Dann folgende Ordner im selben Verzeichnis anlegen:
 - templates (hier kommt home.html hinein)
 - static (hier kommen Bilder use. hinein)
 
Im HTML-File die Links erweitern von
    
    src="images/pic01.JPG"
auf

    src="{% static "images/pic01.JPG" %}"

Dies gilt auf für die css-Dateien und .js-Files am Anfang und Ende der html-Datei. Am Anfang der html-Datei folgendes ergänzen:
    
    {% load static %}
    

Dannn auf der Console:

    python manage.py migrate
    
    python manage.py runserver [ip-Adresse]:8000
    
Über Browser-Eingabe [ip-Adresse]:8000 kann die Web-Page aufgerufen werden - auch von anderen Geräten im Netzwerk.

## Nicht behandelt wird...
### Virtaul Enviroment
Für eine umfangreichere Entwicklung ist der Einsatz einer Virtuellen Umgebung ratsam,jedoch werden wir dies nicht behandeln.

Allgemein bei SW, wie auch bei Python und Bibliotheken, soll immer mit den aktuellsten Versionen gearbeitet werden, somit muss man sich nicht mit bereits gelösten Problemen auseinander setzen. Es macht aber auch Sinn mit älteren Modulen zu arbeiten. Wenn eine eigene Funktion oder Anwendung erstellt wird, so möchte man sicherstellen, dass diese auch in einigen Jahren noch funktioniert ohne weiteren Entwicklungaufwand, d.h. das System und die Pakete, mit welchen entwickelt wurde sollen "konserviert" werden. Dies ist mit Virtual Enviroment (VE) möglich. Das Gesamtsystem kann aktuell gehalten werden und in einem "Verzeichnis" wird ein Entwicklungsstand "konserviert".

### Production Server
Für eine produktiven Einsatz der Web-Page ist ein leistungsfähiger Server wie Apache notwendig. Wir werden diesen jedoch nicht installieren. Wir verwenden für Djanog den Django-internen Entwicklungsserver. 
